In [ ]:
!pip install kagglehub torch torchvision timm streamlit pillow


In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Set permissions


In [ ]:
!kaggle datasets download -d jessicali9530/stanford-dogs-dataset --unzip -p /content/dataset

Dataset URL: https://www.kaggle.com/datasets/jessicali9530/stanford-dogs-dataset
License(s): other
 99% 742M/750M [00:04<00:00, 251MB/s]
100% 750M/750M [00:04<00:00, 194MB/s]


In [ ]:
import torchvision.datasets as datasets

# Load dataset from extracted folder
train_dataset = datasets.ImageFolder(root="/content/dataset/images/Images")

# Extract class names (dog breeds)
classes = train_dataset.classes
num_classes = len(classes)

# Print class details
print(f"Total Dog Breeds: {num_classes}")
print("Example Breeds:", classes[:5])


Total Dog Breeds: 120
Example Breeds: ['n02085620-Chihuahua', 'n02085782-Japanese_spaniel', 'n02085936-Maltese_dog', 'n02086079-Pekinese', 'n02086240-Shih-Tzu']


In [ ]:
import torch
import torch.nn as nn
import timm
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

# Define Image Transformations
transform = transforms.Compose([
    transforms.Resize((384, 384)),  # EfficientNetV2 input size
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Apply transformations to the dataset
# The transform is now applied when the dataset is loaded
train_dataset = datasets.ImageFolder(root="/content/dataset/images/Images", transform=transform)


# Split dataset into training (80%) and testing (20%)
train_size = int(0.8 * len(train_dataset))
test_size = len(train_dataset) - train_size
train_dataset, test_dataset = random_split(train_dataset, [train_size, test_size])

# Data Loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)





In [ ]:
# Load Pretrained Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('tf_efficientnetv2_s', pretrained=True, num_classes=num_classes)
model.to(device)

# Loss Function & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/86.5M [00:00<?, ?B/s]

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = 100 * correct / total
    train_loss = running_loss / len(train_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")


NameError: name 'model' is not defined

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=classes)

    print(f"Test Accuracy: {acc * 100:.2f}%")
    print("Classification Report:\n", report)

# Run evaluation
evaluate_model(model, test_loader)


In [ ]:
model_save_path = "/content/dog_breed_classifier.pth"
torch.save(model.state_dict(), model_save_path)
print("Model saved to:", model_save_path)


In [ ]:
# Download the model
from google.colab import files
files.download(model_save_path)